![](./img/dl_banner.jpg)

# pytorch循环神经网络
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

## 1.导入工具库

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 2.设定超参数

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 有gpu就用gpu
sequence_length = 28 # 序列长度
input_size = 28 # 每个step上输入的数据维度
hidden_size = 128 # 隐层大小
num_layers = 2 # 层次
num_classes = 10 # 总类别数
batch_size = 100 # 一批数据大小
num_epochs = 2 # 总迭代轮次
learning_rate = 0.01 # 学习率

## 3.准备数据
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [3]:
# MNIST数据集
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader构建输入流水线
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## 4.构建模型、定义损失与优化器
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # 设定初始的hidden和cell state 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # LSTM前向运算
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # 取出最后的输出进行全连接分类
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# 交叉熵损失与Adam优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 5.迭代与训练
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [5]:
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # 前向运算
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('目前总轮次进程 [{}/{}], 当前批次进程 [{}/{}], 当前损失: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

目前总轮次进程 [1/2], 当前批次进程 [100/600], 当前损失: 0.5841
目前总轮次进程 [1/2], 当前批次进程 [200/600], 当前损失: 0.2581
目前总轮次进程 [1/2], 当前批次进程 [300/600], 当前损失: 0.1444
目前总轮次进程 [1/2], 当前批次进程 [400/600], 当前损失: 0.0718
目前总轮次进程 [1/2], 当前批次进程 [500/600], 当前损失: 0.3076
目前总轮次进程 [1/2], 当前批次进程 [600/600], 当前损失: 0.1950
目前总轮次进程 [2/2], 当前批次进程 [100/600], 当前损失: 0.1937
目前总轮次进程 [2/2], 当前批次进程 [200/600], 当前损失: 0.0336
目前总轮次进程 [2/2], 当前批次进程 [300/600], 当前损失: 0.1266
目前总轮次进程 [2/2], 当前批次进程 [400/600], 当前损失: 0.0568
目前总轮次进程 [2/2], 当前批次进程 [500/600], 当前损失: 0.1283
目前总轮次进程 [2/2], 当前批次进程 [600/600], 当前损失: 0.1044


## 6.预估与测试集评估
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [7]:
model.eval()  # 评估模式
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('在10000张测试图片上的准确率为: {} %'.format(100 * correct / total)) 

# 模型存储
torch.save(model.state_dict(), './model/rnn.ckpt')

在10000张测试图片上的准确率为: 97.95 %


### 版权归 © 稀牛学院 所有 保留所有权利
![](./img/xiniu_neteasy.png)